Digital daydreaming in latent space with CLIP Interogator and Stable Diffusion

In [ ]:
!nvidia-smi -L

In [ ]:
!pip install diffusers==0.2.4
#!pip install -qq -U diffusers==0.3.0

In [ ]:
import subprocess

def setup():
    install_cmds = [
        #['pip', 'install', '-e', 'git+https://github.com/openai/CLIP.git@main#egg=clip'],
        ['pip', 'install', '-e', 'git+https://github.com/MirageML/BLIP.git@main#egg=blip'],#  https://github.com/pharmapsychotic/BLIP.git@lib#egg=blip'],
        ['git', 'clone', 'https://github.com/pharmapsychotic/clip-interrogator.git'],
        ['pip', 'install', 'open_clip_torch', 'ftfy', 'gradio', 'regex', 'tqdm', 'transformers==4.24.0', 'timm', 'fairscale', 'requests']        
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

import sys
sys.path.append('src/blip')
sys.path.append('src/clip')
sys.path.append('clip-interrogator')

import gradio as gr
from clip_interrogator import Config, Interrogator

ci = Interrogator(Config())

def inference(image, mode):
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    else:
        return ci.interrogate_fast(image)


In [ ]:
!mkdir spun
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/stub_materials/figor.png /content/st.png #your seed image
!cp /content/drive/MyDrive/stub_materials/image_to_image.py /content/

In [ ]:
# don't forget:
# huggingface-cli login
# I always keep my token # here

import os
import argparse
import torch
import numpy as np
from diffusers import StableDiffusionPipeline
from PIL import Image
from io import BytesIO
from image_to_image import StableDiffusionImg2ImgPipeline, preprocess
#from diffusers.models import AutoencoderKL
class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)
from torch import autocast
rond=np.random.randint(7,100000000)
generator = torch.Generator("cuda").manual_seed(rond)
print( "random seed " + str(rond))
device = "cuda"
#vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    #vae=vae,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

pipe.safety_checker = FakeSafety()
file1 = open("dialog1.txt", "a")
for xn in range(0,250,1):
  im= Image.open("st.png").convert("RGB")
  im=im.resize((512,512))
  cliprompt=ci.interrogate_fast(im)
  print(cliprompt)
  file1.write(cliprompt+"\n")
  init_image = preprocess(im)
  with autocast("cuda"):
    image = pipe(cliprompt, init_image=init_image, strength=0.65, guidance_scale=7.0, num_inference_steps=20, generator=generator)["sample"][0]  # image
  image.save("st.png")
  image.save("spun2/"+str(xn).zfill(4)+".png")
  os.system("bash 1julien.sh st.png zi")
file1.close()
!cp dialog1.txt spun2/
!zip -r 11_29_22_spun2.zip spun2/ 
!cp 11_29_22_spun2.zip /content/drive/MyDrive/product

In [ ]:
from google.colab import runtime
runtime.unassign()